In [1]:
import os
import pandas as pd
import numpy as np
import pandas as pd
from pandas import json_normalize
from pandas.tseries.offsets import DateOffset
import numpy as np
import bisect
import math
import statistics
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
from itertools import combinations
import urllib.request, json
import csv
from pandas import json_normalize
import math
import scipy.stats as sp
# The first thing we want to do is import the Pandas library and set the filepath to our data file
import warnings
from pandas.tseries.offsets import DateOffset
import warnings
import requests
import datetime
import seaborn as sns
import time
import statistics
import matplotlib.pyplot as plt
import urllib.request, json

import seaborn as sns
from scipy.signal import argrelextrema
from collections import deque
from itertools import groupby, chain
from collections import OrderedDict
import os
import datetime as dt
from datetime import datetime, date, timedelta
import warnings
import re


import sklearn
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier, RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier


import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

from itertools import groupby, chain
from collections import OrderedDict


def get_data_yfinance(asset, interval='1d', start='2011-01-01'):
    date_column = 'Date'
    list_of_days = ['1d', '5d', '1wk', '1mo', '3mo']
    list_of_minutes = ['1m', '2m', '5m', '15m', '30m', '60m', '90m']
    df = yf.download(asset, start=start, threads= False, interval=interval)
    df = df.reset_index()
    if interval== '1h':
        df = df.rename(columns={'index':'Date'})
    elif interval in list_of_minutes:
        df = df.rename(columns={'Datetime':'Date'})
        #df[date_column] = df[date_column].str[:]

    # Now that we have loaded our data into the dataframe, we can preview it using the print & .head() function

    df[date_column] = pd.to_datetime(df[date_column])
    df = df.sort_values([date_column], ascending=[True]).reset_index(drop=True).sort_values([date_column], ascending=[False])
    df = df.rename(columns={'Date':'date','Open':'open', 'High':'high', 'Low': 'low','Close':'close', 'Volume':'tradecount'})
    df['asset'] = asset
    if verificar_valores_sabados_domingos(df)==False:
        df = actualizar_dataframe_para_findesemana(df)
    return(df)

def get_prices_indicators_data(ticker, date_column='date'):
    df = get_data_yfinance(ticker)[['date', 'open', 'high', 'low', 'close']]

    df_unsorted = df.sort_values([date_column], ascending=[False]).reset_index(drop=True).sort_values([date_column], ascending=[True])
    df_direction = target(df_unsorted).direction()
    df_direction['target'] = df_direction['direction'].apply(lambda x: 1 if x> 0 else 0)
    df_total = pd.merge(df,df_direction[['date', 'target']], on='date', how='inner')
    df_total['symbol']=ticker
    # 2 obtener indicadores y osciladores
    #INDICATORS
    df_squeez = squeez_momentum_indicator(df)
    df_squeez = df_squeez.rename(columns={'value':'value_squeez'})
    df_squeez = df_squeez.dropna()
    df_squeez = df_squeez.replace({True: 1, False: 0})
    list_of_squeez_indicator = get_list_of_fields(df_squeez)
    df_ind_osc, dictionary_of_indicators_oscilators = indicators_oscilators(df_unsorted)
    df_ind_osc =df_ind_osc.dropna()
    dictionary_of_indicators_oscilators['squeez'] = list_of_squeez_indicator
    df_ind_osc_target = pd.merge(df_total, df_ind_osc, on='date', how='inner')
    return df_ind_osc_target


class target:
    def __init__(self,df,column_date='date'):
        self.df = df
        self.column_date = column_date

    def max_min(self, initial_days=0, days=10, column_target='high', method=max):
        df = self.df
        column_date = self.column_date

        df_target = df[[column_date,column_target]].copy()
        df_target_original = df_target.copy()
        list_of_targets=[column_date]
        #in loop add one column per day
        for i in range(initial_days,days+1):
            df_target_temp = df_target_original.copy()
            df_target_temp[column_date+str(i)] = df_target_temp[column_date] + pd.Timedelta(days=-i)
            df_target_temp = df_target_temp[[column_date+str(i),column_target]].rename(columns={column_target:column_target+str(i)})
            df_target=pd.merge(df_target,df_target_temp, how='left',left_on=column_date, right_on=column_date+str(i))
            df_target = df_target.dropna()
            list_of_targets.append(column_target+str(i))

        df_target = df_target[list_of_targets]
        if method==max:
            df_target['target'] = df_target[list_of_targets].max(axis=1)
        if method==min:
            df_target['target'] = df_target[list_of_targets].min(axis=1)
        df_target=df_target[[column_date, 'target']]
        df_target = df_target.sort_values([column_date], ascending=[True]).reset_index(drop=True).sort_values([column_date], ascending=[False])

        return(df_target)

    def close(self, days=10,column_target='close'):
        df = self.df
        column_date = self.column_date
        df_target = df[[column_date,column_target]].copy()
        df_target[column_date+ "_-_" +str(days)] = df_target[column_date] + pd.Timedelta(days=-days)
        df_target = df_target.drop(columns=column_date, axis=1)
        dictionary={column_date+ "_-_" +str(days):column_date, column_target:'target'}
        df_target = df_target.rename(columns=dictionary)
        df_target = df_target[[column_date, 'target']]
        return (df_target)

    def direction(self,days=10,column_target='close', column_high='high', column_low='low'):
        df = self.df
        column_date = self.column_date
        df_close = df[[column_date, 'close']].copy()
        df_target_max = target(df).max_min(method=max).rename(columns={'target':'max'})
        df_target_min = target(df).max_min(method=min, column_target='low').rename(columns={'target':'min'})
        df_target_close = target(df).close().rename(columns={'target':'target_close'})
        df_final = pd.merge(df_close, df_target_max, on=column_date, how='inner')
        df_final = pd.merge(df_final, df_target_min, on=column_date, how='inner')
        df_final = pd.merge(df_final, df_target_close, on=column_date, how='inner')
        #df_final['direction'] = df_final['close'] + (df_final['max'] - df_final['close']  + df_final['min'] - df_final['close'] + df_final['target_close'] - df_final['close'])/3
        df_final['direction'] = ((df_final['max']- df_final['close'])/2 + (df_final['min']- df_final['close'])/2 )/2 + (df_final['target_close'] - df_final['close'])/2
        #df_final = df_final[['date','direction']]

        return df_final
def verificar_valores_sabados_domingos(df):
    # Asegúrate de que la columna "date" esté en formato de fecha
    df['date'] = pd.to_datetime(df['date'])

    # Obtiene el día de la semana para cada fecha (0 para lunes, 1 para martes, etc.)
    df['dayofweek'] = df['date'].dt.dayofweek

    # Filtra las filas que corresponden a sábados (5) o domingos (6)
    sabados_domingos = df[(df['dayofweek'] == 5) | (df['dayofweek'] == 6)]

    # Si "sabados_domingos" está vacío, significa que no hay valores para sábados ni domingos en el DataFrame
    if sabados_domingos.empty:
        weekend_good = False
    else:
        weekend_good = True
    return weekend_good


def actualizar_dataframe_para_findesemana(df):
    # Asegúrate de que la columna "date" esté en formato de fecha
    df['date'] = pd.to_datetime(df['date'])

    # Agrega una columna "week_day" que contenga el día de la semana (0 para lunes, 1 para martes, etc.)
    df['week_day'] = df['date'].dt.dayofweek

    # Crea un nuevo DataFrame para almacenar los datos actualizados
    new_df = pd.DataFrame(columns=df.columns)

    # Recorre el DataFrame original
    for index, row in df.iterrows():
        new_df = new_df.append(row)

        # Si el día de la semana es viernes (4)
        if row['week_day'] == 4:
            # Agrega una fila para el sábado (5)
            new_row_sabado = row.copy()
            new_row_sabado['date'] = row['date'] + timedelta(days=1)
            new_row_sabado['week_day'] = 5
            new_df = new_df.append(new_row_sabado)

            # Agrega una fila para el domingo (6)
            new_row_domingo = row.copy()
            new_row_domingo['date'] = row['date'] + timedelta(days=2)
            new_row_domingo['week_day'] = 6
            new_df = new_df.append(new_row_domingo)

    # Restablece el índice del nuevo DataFrame
    new_df.reset_index(drop=True, inplace=True)

    return new_df

def add_derivates(df, feature, periods=1):
    df_new = df.copy()
    diff_1 = df[[feature]].diff(periods=periods).rename(columns={feature: feature +"_diff1"})
    df_new = pd.concat([df_new, diff_1], axis=1).dropna()
    del df
    return df_new

def get_ribbons(df, sma_low, sma_high, list_of_features, date_column='date'):
    df_out=df.copy()
    list_of_columns = [date_column] + list_of_features
    df_out=df_out[list_of_columns]
    list_of_sma = []
    list_of_ribbon = []
    for i in list_of_features:
        sma_low_ = i + '_sma_' + str(sma_low)
        list_of_sma.append(sma_low_)
        df_out[sma_low_] = pd.Series.to_frame(ta.sma(df_out[i], length=sma_low))
        sma_high_ = i + '_sma_' + str(sma_high)
        list_of_sma.append(sma_high_)
        df_out[sma_high_] = pd.Series.to_frame(ta.sma(df_out[i], length=sma_high))
        ribbon_ = 'ribbon_' + i + '_' +  str(sma_low) + '_' + str(sma_high)
        list_of_ribbon.append(ribbon_)
        df_out[ribbon_] = (df_out[sma_low_] - df_out[sma_high_])
    df_out = df_out.dropna()
    return df_out


#########################################################

def grouper(_lst, interval=0.04):
    z = zip(_lst,_lst[1:])
    return [OrderedDict.fromkeys(chain.from_iterable(g)).keys() for k,g in groupby(z,key=lambda x:x[1]-x[0]<interval) if k]

def supports_and_resistences(df, interval=0.04, column_close='close', column_high='high', column_low='low', \
                             date_column='date', look_back=200, window=20):
    df_new = df.copy()
    iters = int(look_back/window)
    data_df = {date_column: '', 'second_last_support':[''], 'last_support':[''], 'last_resistence':[''], 'second_last_resistence': ['']}
    df_supports_and_resistances = pd.DataFrame(data=data_df)
    df_supports_and_resistances = df_supports_and_resistances[0:0]
    #count=0

    # Take resistances only until the lookback normally 200
    for ii in range(0,len(df)-look_back):
        value = df[column_close].iloc[ii]
        date = df[date_column].iloc[ii]
        df_temp=df_new[ii:look_back+ii]
        list_of_points=[]

        for it in range(0,iters):
            df_small = df_temp[window*it:window*it+window]
            max_high = df_small[column_high].max()
            max_close = df_small[column_close].max()
            min_low = df_small[column_low].min()
            min_close = df_small[column_close].min()
            list_of_new_points = [max_high, max_close, min_low, min_close]
            list_of_points += list_of_new_points
            list_of_points = sorted(list_of_points)
        new_list = grouper(list_of_points, value * interval)
        list_of_final_points=[]

        for j in range(len(new_list)):
            list_of_final_points.append(statistics.mean(new_list[j]))
        list_of_close_points=calculate_diferentials_resistances(list_of_final_points, value)
        #create dataframe
        d = {date_column: [date], 'second_last_support':[list_of_close_points[0]], 'last_support': [list_of_close_points[1]], \
             'last_resistence': [list_of_close_points[2]], 'second_last_resistence': [list_of_close_points[3]]}
        df_temporal = pd.DataFrame(data=d)
        df_supports_and_resistances = pd.concat([df_supports_and_resistances, df_temporal], axis=0)
    return(df_supports_and_resistances)

def get_list_of_fields(df, date_column='date', *args):
    list_of_fields = list(df.columns)
    if date_column in list_of_fields:
        list_of_fields.remove(date_column)
    if len(args)>0:
        for new_column_to_delete in args:
            if new_column_to_delete in list_of_fields:
                list_of_fields.remove(new_column_to_delete)
    return list_of_fields

def supports_and_resistences_more_days(df,days_back, window_=20):
    df_sr = supports_and_resistences(df, look_back=days_back, window=window_)
    df_sr = df_sr.rename(columns={'second_last_support':'second_last_support_'+ str(days_back),
                          'last_support': 'last_support_'+ str(days_back),
                          'last_resistence': 'last_resistence_'+ str(days_back),
                          'second_last_resistence':'second_last_resistence_'+ str(days_back)})
    return df_sr

def get_fibonacci(df,days=200, column_close='close', column_high='high', column_low='low', date_column='date', list_of_points=[0, 0.236, 0.382, 0.5, 0.618, 0.786, 1, 1.278, 1.414]):
    list_of_points=list_of_points
    def calculate_fibonacci_levels(price_max, price_min, list_of_points):
        diff = price_max-price_min
        fibo_levels = [(price_min + i *diff) for i in list_of_points]
        return(fibo_levels)

    data_df = {date_column: '', 'second_fibo_last_support':[''], 'last_fibo_support':[''], 'last_fibo_resistence':[''], 'second_last_fibo_resistence': ['']}
    df_supports_and_resistances = pd.DataFrame(data=data_df)
    df_supports_and_resistances = df_supports_and_resistances[0:0]

    for ii in range(0,len(df)-days):
        df_temp=df[ii:days+ii].copy()
        value = df[column_close].iloc[ii]
        date = df[date_column].iloc[ii]
        price_max = df_temp[column_high].max()
        price_min = df_temp[column_low].min()
        list_of_final_points=calculate_fibonacci_levels(price_max, price_min, list_of_points)
        list_of_close_points=calculate_diferentials_resistances(list_of_final_points, value)
        #create dataframe
        d = {'date': [date], 'second_fibo_last_support':[list_of_close_points[0]], 'last_fibo_support': [list_of_close_points[1]], 'last_fibo_resistence': [list_of_close_points[2]], 'second_last_fibo_resistence': [list_of_close_points[3]]}
        df_temporal = pd.DataFrame(data=d)
        df_supports_and_resistances = pd.concat([df_supports_and_resistances, df_temporal], axis=0)

    for ii in range(len(df)-days,len(df)):
        df_temp=df[ii:len(df)].copy()
        value = df[column_close].iloc[ii]
        date = df[date_column].iloc[ii]
        price_max = df_temp[column_high].max()
        price_min = df_temp[column_low].min()
        list_of_final_points=calculate_fibonacci_levels(price_max, price_min, list_of_points)
        list_of_close_points=calculate_diferentials_resistances(list_of_final_points, value)
        #create dataframe
        d = {date_column: [date], 'second_fibo_last_support':[list_of_close_points[0]], 'last_fibo_support': [list_of_close_points[1]], 'last_fibo_resistence': [list_of_close_points[2]], 'second_last_fibo_resistence': [list_of_close_points[3]]}
        df_temporal = pd.DataFrame(data=d)
        df_supports_and_resistances = pd.concat([df_supports_and_resistances, df_temporal], axis=0)


    #return(list(df_supports_and_resistances.columns),df_supports_and_resistances)
    return(df_supports_and_resistances)

def get_fibonacci_levels(df):
    df_close = df[['date', 'close']]
    df_fibo = get_fibonacci(df)
    df_fibo_to_clean = pd.merge(df_close, df_fibo, on='date', how='left').replace('NOT FOUND', -1)
    df_fibo_filled = fill_support_not_found_values_in_sr(df_fibo_to_clean, 'last_fibo_support')
    df_fibo_filled = fill_support_not_found_values_in_sr(df_fibo_filled, 'second_fibo_last_support', measure_column='last_fibo_support')
    minimum_value = df_fibo_filled.close.min()
    df_fibo_filled = df_fibo_filled.replace({'last_fibo_support': {-1: minimum_value},\
                           'second_fibo_last_support': {-1: minimum_value}})
    df_fibo_filled = df_fibo_filled.drop("close", axis='columns')

    return(df_fibo_filled)

def obtain_list_of_volumen_areas(df, column_close='close', volume_column='tradecount', aggregation=1000):
    agg = int(df[column_close].max()/aggregation)
    if agg==0:
        agg=1
    data_vp = df[[column_close, volume_column]].copy()
    data_vp[column_close] = data_vp[column_close].apply(lambda x: int(agg * round(float(x)/agg)))
    data_vp = data_vp.set_index(column_close)
    data_vp = data_vp.groupby([column_close]).sum()
    df_areas = data_vp.sort_values(by=volume_column, ascending=False).reset_index()
    # take only the quartil
    mean_of_df = df_areas[['tradecount']].mean()
    max_of_df = df_areas[['tradecount']].max()
    std_of_df = df_areas[['tradecount']].std()
    threshold = mean_of_df - std_of_df
    df_temp_areas = df_areas[df_areas['tradecount']>threshold[0]]
    #convert_to_list
    list_of_areas = df_temp_areas[column_close].tolist()
    list_of_areas=sorted(list_of_areas)
    new_list = grouper(list_of_areas, interval=int(df[column_close].max()/(len(df)/5)))
    list_of_final_points = [statistics.mean(new_list[j]) for j in range(len(new_list))]
    return(list_of_final_points)

def get_area_volume(df, column_close='close', volume_column='tradecount', date_column='date'):
    data_df = {date_column: '', 'second_area_last_support':[''], 'last_area_support':[''], 'last_area_resistence':[''], 'second_area_resistence': ['']}
    df_supports_and_resistances = pd.DataFrame(data=data_df)
    df_supports_and_resistances = df_supports_and_resistances[0:0]
    for i in range(0,len(df)):
        df_temp = df[i:len(df)]
        value = df[column_close].iloc[i]
        date = df[date_column].iloc[i]
        list_of_final_points = obtain_list_of_volumen_areas(df_temp)
        list_of_close_points=calculate_diferentials_resistances(list_of_final_points, value)
        #create dataframe
        d = {date_column: [date], 'second_area_last_support':[list_of_close_points[0]], 'last_area_support': [list_of_close_points[1]], 'last_area_resistence': [list_of_close_points[2]], 'second_area_resistence': [list_of_close_points[3]]}
        df_temporal = pd.DataFrame(data=d)
        df_supports_and_resistances = pd.concat([df_supports_and_resistances, df_temporal], axis=0)
    return(df_supports_and_resistances)

def get_volume_areas(df, date_column='date', close_column='close'):
    df_close = df[[date_column, close_column]]
    df_areas = get_area_volume(df)
    df_areas_and_close = pd.merge(df_close, df_areas, on='date', how='inner').replace('NOT FOUND', -1).replace('UNKNOWN', -1)
    df_filled = fill_support_not_found_values_in_sr(df_areas_and_close, 'last_area_support')
    df_filled = fill_support_not_found_values_in_sr(df_filled, 'second_area_last_support', measure_column='last_area_support')
    df_filled = fill_resistence_not_found_values_in_sr(df_filled, 'last_area_resistence')
    df_filled = fill_resistence_not_found_values_in_sr(df_filled, 'second_area_resistence', measure_column='last_area_resistence')
    df_out = df_filled.copy()
    for i in range(len(df_out)):
        df_it_out = df_out[i:len(df_out)]
        maximum_value = df_it_out.close.max()
        minimum_value = df_it_out.close.min()
        if df_out[['last_area_support']].iloc[i][0]==-1:
            df_out.at[i, 'last_area_support'] = minimum_value
        if df_out[['second_area_last_support']].iloc[i][0]==-1:
            df_out.at[i, 'second_area_last_support'] = minimum_value
        if df_out[['last_area_resistence']].iloc[i][0]==-1:
            df_out.at[i, 'last_area_resistence'] = maximum_value
        if df_out[['second_area_resistence']].iloc[i][0]==-1:
            df_out.at[i, 'second_area_resistence'] = maximum_value
    df_out =df_out.drop(columns=close_column, axis=1)
    return df_out

def pivot_cpr(df, high='high', close='close', low='low', agg='M'):
    df = df.copy()
    df['Pivot'+agg] = (df[high] + df[low] + df[close])/3
    df['BC'+agg] = (df[high] + df[low])/2
    df['TC'+agg] = (df['Pivot'+agg] - df['BC'+agg]) + df['Pivot'+agg]
    df['R1'+agg] = 2*df['Pivot'+agg] - df[low]
    df['R2'+agg] = df['Pivot'+agg] + (df[high] - df[low])
    df['R3'+agg] = df['R1'+agg] + (df[high] - df[low])
    #df['R4'+agg] = df['R3'+agg] + (df['R2'+agg] - df['R1'+agg])
    df['S1'+agg] = 2*df['Pivot'+agg] - df[high]
    df['S2'+agg] = df['Pivot'+agg] - (df[high] - df[low])
    df['S3'+agg] = df['S1'+agg] - (df[high] - df[low])
    #df['S4'+agg] = df['S3'+agg] - (df['S1'+agg] - df['S2'+agg])
    return(df)

def index_timeframe(df, aggregation='W', date_column='date'):
    df2 = df.copy()
    df2[date_column] = pd.to_datetime(df[date_column], format='%Y-%m-%d')
    df2.set_index(date_column, inplace=True)
    df2.sort_index(inplace=True)
    tradecount = df2[['tradecount']].resample(aggregation).mean()
    high = df2[['high']].resample(aggregation).max()
    low = df2[['low']].resample(aggregation).min()
    open_ = df2[['open']].resample(aggregation).first()
    close = df2[['close']].resample(aggregation).last()
    #Volume_asset = df2[['Volume asset']].resample(aggregation).mean()
    #Volume_pair = df2[['Volume pair']].resample(aggregation).mean()

    list_of_fields = ['date_'+str(aggregation),'open_'+str(aggregation), 'high_'+str(aggregation), 'low_'+str(aggregation), 'close_'+str(aggregation), 'Volume_BTC_'+str(aggregation), 'Volume_USDT_'+str(aggregation), 'tradecount_'+str(aggregation), 'date_join_'+str(aggregation)]
    list_of_df = [high, low, close, tradecount]

    #list_of_columns = [date_column, 'open', 'high', 'low', 'close', 'Volume asset', 'Volume pair', 'tradecount', 'date_join']

    df_final = open_
    for i in list_of_df:
        df_final = pd.concat([df_final,i], axis=1)
    df_final = df_final.reset_index()
    df_final[date_column] = pd.to_datetime(df_final[date_column],format='%Y-%m-%d')
    df_final = df_final.sort_values(['date'], ascending=[True]).reset_index(drop=True).sort_values(['date'], ascending=[False])


    if aggregation=='W':
        alfa=7
    elif aggregation=='M':
        alfa=28
    df_final['date_join'] = pd.to_datetime(df_final.date) + pd.to_timedelta(+alfa, unit="D")

    #df_final = indicators_oscilators(df_final)


    list_of_fields = [(str(df_final.columns[i])+ "_"+aggregation) for i in range(len(df_final.columns))]
    list_of_columns = df_final.columns
    dictionary_columns = zip(list_of_columns, list_of_fields)
    dictionary = dict(dictionary_columns)
    df_final.rename(dictionary, axis=1, inplace=True)
    df_final = df_final.tail(len(df_final)-1)

    #df_final = df_final.dropna()

    return(df_final)

def get_monthly_cpr(df, column_date_day ='date',column_close='close', volume_column='tradecount', column_open='open',
                      column_high='high', column_low='low'):
    aggregation='M'
    df_d = df.copy()
    df_d['year'] = df_d[column_date_day].dt.year
    df_d['week_day']=df_d[column_date_day].dt.dayofweek
    df_d['week_num']=df_d[column_date_day].dt.isocalendar().week
    df_d['month_num'] = df_d[column_date_day].dt.month

    #monthly variables
    df_m = index_timeframe(df, aggregation=aggregation)
    date_m = column_date_day + '_' + aggregation
    date_join = column_date_day + '_join_' + aggregation
    open_m = column_open + '_' + aggregation
    close_m = column_close + '_' + aggregation
    high_m = column_high + '_' + aggregation
    low_m = column_low + '_' + aggregation
    #variables to merge
    df_m = df_m[[date_m, date_join, open_m, high_m , low_m, close_m]]
    df_m_pivot = pivot_cpr(df_m, high=high_m, close=close_m, low=low_m)
    df_m_pivot['year'] = df_m_pivot[date_join].dt.year
    df_m_pivot['month_num'] = df_m_pivot[date_join].dt.month
    df_m_pivot = df_m_pivot.drop(columns=[date_m, date_join], axis=1)
    #merge
    df_cpr = pd.merge(df_d,df_m_pivot, how='left', on=['year','month_num'])

    data_df = {column_date_day: '', 'second_cpr_last_support':[''], 'last_cpr_support':[''], 'last_cpr_resistence':[''], 'second_last_cpr_resistence': ['']}
    df_supports_and_resistances = pd.DataFrame(data=data_df)
    df_supports_and_resistances = df_supports_and_resistances[0:0]

    for i in range(len(df_cpr)):
        list_of_points = df_cpr[['PivotM', 'BCM', 'TCM', 'R1M', 'R2M', 'R3M', 'S1M', 'S2M', 'S3M']].iloc[i]
        list_of_points = sorted(list_of_points)
        value=df_cpr[column_close].iloc[i]
        date=df_cpr[column_date_day].iloc[i]
        list_of_close_points=calculate_diferentials_resistances(list_of_points, value)
        #create dataframe
        d = {column_date_day: [date], 'second_cpr_last_support':[list_of_close_points[0]], 'last_cpr_support': [list_of_close_points[1]], 'last_cpr_resistence': [list_of_close_points[2]], 'second_last_cpr_resistence': [list_of_close_points[3]]}
        df_temporal = pd.DataFrame(data=d)
        df_supports_and_resistances = pd.concat([df_supports_and_resistances, df_temporal], axis=0)

    return(df_supports_and_resistances)

def get_monthly_cpr_points(df, date_column='date', close_column='close'):
    df_close = df[[date_column, close_column]].copy()
    df_cpr = get_monthly_cpr(df)
    df_cpr_and_close = pd.merge(df_close, df_cpr, on='date', how='inner').replace('NOT FOUND', -1).replace('UNKNOWN', -1)
    df_filled = fill_support_not_found_values_in_sr(df_cpr_and_close, 'last_cpr_support')
    df_filled = fill_support_not_found_values_in_sr(df_filled, 'second_cpr_last_support', measure_column='last_cpr_support')
    df_filled = fill_resistence_not_found_values_in_sr(df_filled, 'last_cpr_resistence')
    df_filled = fill_resistence_not_found_values_in_sr(df_filled, 'second_last_cpr_resistence', measure_column='last_cpr_resistence')
    df_out = df_filled.dropna().copy()
    for i in range(len(df_out)):
        df_it_out = df_out[i:len(df_out)]
        maximum_value = df_it_out.close.max()
        minimum_value = df_it_out.close.min()
        if df_out[['last_cpr_support']].iloc[i][0]==-1:
            df_out.at[i, 'last_cpr_support'] = minimum_value
        if df_out[['second_cpr_last_support']].iloc[i][0]==-1:
            df_out.at[i, 'second_cpr_last_support'] = minimum_value
        if df_out[['last_cpr_resistence']].iloc[i][0]==-1:
            df_out.at[i, 'last_cpr_resistence'] = maximum_value
        if df_out[['second_last_cpr_resistence']].iloc[i][0]==-1:
            df_out.at[i, 'second_last_cpr_resistence'] = maximum_value
    df_out =df_out.drop(columns=close_column, axis=1)
    return(df_out)

def get_fear_and_greed():
    url = 'https://api.alternative.me/fng/?limit=0&format=json'
    with urllib.request.urlopen(url) as url:
        data = json.loads(url.read().decode())
    df = json_normalize(data['data'])
    df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
    df = df.drop(columns = ['time_until_update', 'value_classification'])
    df = df.rename(columns={'timestamp':'date'})
    df = df[['date', 'value']]
    return df


#######################################################################


def indicators_oscilators(df):
    dictionary_of_features = {}
    initial_columns=list(df.columns)
    initial_columns = initial_columns[1:]
    #df = df.sort_index(ascending=True)
    list_of_indicators = []
    # INDICATORS #  queda pendiente revisar el span

    try:
        willr = pd.Series.to_frame(ta.willr(df['high'],df['low'],df['close']))
        list_of_indicators.append(willr)
        dictionary_of_features["willr"] = get_list_of_fields(willr)
    except:
        print("william no da valores")
    try:
        sma20 = pd.Series.to_frame(ta.sma(df["close"], length=20))
        list_of_indicators.append(sma20)
        dictionary_of_features["sma20"] = get_list_of_fields(sma20)
    except:
        print("sma20 no da valores")
    try:
        ema20_ohlc4 = pd.Series.to_frame(df.ta.ema(close=df.ta.ohlc4(), length=20, suffix="OHLC4"))
        list_of_indicators.append(ema20_ohlc4)
        dictionary_of_features["ema20"] = get_list_of_fields(ema20_ohlc4)
    except:
        print("ema20_ohlc4 no da valores")
    try:
        sma55 = pd.Series.to_frame(ta.sma(df["close"], length=55))
        list_of_indicators.append(sma55)
        dictionary_of_features["sma55"] = get_list_of_fields(sma55)
    except:
        print("sma55 no da valores")
    try:
        ema55_ohlc4 = pd.Series.to_frame(df.ta.ema(close=df.ta.ohlc4(), length=55, suffix="OHLC4"))
        list_of_indicators.append(ema55_ohlc4)
        dictionary_of_features["ema55"] = get_list_of_fields(ema55_ohlc4)
    except:
        print("ema55_ohlc4 no da valores")

    try:
        ema200_ohlc4 = pd.Series.to_frame(df.ta.ema(close=df.ta.ohlc4(), length=200, suffix="OHLC4"))
        list_of_indicators.append(ema200_ohlc4)
        dictionary_of_features["ema200"] = get_list_of_fields(ema200_ohlc4)
    except:
        print("ema200_ohlc4 no da valores")

    try:
        ichimoku, span = ta.ichimoku(df["high"], df["low"], df["close"]) #review span
        ichimoku = ichimoku.drop(columns=['ICS_26'])
        list_of_indicators.append(ichimoku)
        dictionary_of_features["ichimoku"] = get_list_of_fields(ichimoku)
    except:
        print("ichimoku no da valores")
    try:
        adx = ta.adx(df["high"], df["low"], df["close"])
        list_of_indicators.append(adx)
        dictionary_of_features["adx"] = get_list_of_fields(adx)
    except:
        print("adx no da valores")
    try:
        bbands = ta.bbands(df["close"])
        list_of_indicators.append(bbands)
        dictionary_of_features["bbands"] = get_list_of_fields(bbands)
    except:
        print("bbands no da valores")

    # OSCILATORS #
    list_of_oscilators = []

    try:
        atr = pd.Series.to_frame(ta.atr(df['high'],df['low'],df['close']))
        list_of_oscilators.append(atr)
        dictionary_of_features["atr"] = get_list_of_fields(atr)
    except:
        print("momentum no da valores")
    try:
        momentum = ta.squeeze(df["high"], df["low"], df["close"])
        list_of_oscilators.append(momentum)
        dictionary_of_features["momentum "] = get_list_of_fields(momentum )
    except:
        print("momentum no da valores")
    try:
        rsi = pd.Series.to_frame(ta.rsi(df["close"]))
        list_of_oscilators.append(rsi)
        dictionary_of_features["rsi"] = get_list_of_fields(rsi)
    except:
        print("rsi no da valores")
    try:
        rsi_6 = pd.Series.to_frame(ta.rsi(df["close"], length=6))
        list_of_oscilators.append(rsi_6)
        dictionary_of_features["rsi_6"] = get_list_of_fields(rsi_6)
    except:
        print("rsi 6 no da valores")
    try:
        rsi_12 = pd.Series.to_frame(ta.rsi(df["close"], length=12))
        list_of_oscilators.append(rsi_12)
        dictionary_of_features["rsi_12"] = get_list_of_fields(rsi_12)
    except:
        print("rsi 12 no da valores")
    try:
        rsi_24 = pd.Series.to_frame(ta.rsi(df["close"], length=24))
        list_of_oscilators.append(rsi_24)
        dictionary_of_features["rsi_24"] = get_list_of_fields(rsi_24)
    except:
        print("rsi 24  no da valores")
    try:
        macd = ta.macd(df["close"])
        list_of_oscilators.append(macd)
        dictionary_of_features["macd"] = get_list_of_fields(macd)
    except:
        print("macd no da valores")
    try:
        kdj = ta.kdj(df["high"], df["low"], df["close"])
        list_of_oscilators.append(kdj)
        dictionary_of_features["kdj"] = get_list_of_fields(kdj)
    except:
        print("kdj  no da valores")

    for indicator in list_of_indicators:
        df = pd.concat([df,indicator],axis=1)
    for oscilator in list_of_oscilators:
        df = pd.concat([df,oscilator],axis=1)

    df = df.sort_index(ascending=False)

    list_of_columns = df.columns
    for column in list_of_columns:
        if df[column].isnull().all() == True:
            df = df.drop(columns=column, axis=1)
    df = df.drop(columns=initial_columns, axis=1)

    return(df, dictionary_of_features)


#########################################################


def get_rate_volume(filepath_head, filepath_tail, asset, pair):
    #filepath_head = "https://www.cryptodatadownload.com/cdd/Binance_"
    #filepath_tail = "_d.csv"
    #filepath = "https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_d.csv"
    # Now we want to create a dataframe and use Pandas' to_csv function to read in our file
    filepath2 = filepath_head + '%s%s' % (asset,pair) +  filepath_tail
    df = pd.read_csv(filepath2, skiprows=1)  # we use skiprows parameter because first row contains our web address
    # Now that we have loaded our data into the dataframe, we can preview it using the print & .head() function
    df['date'] = df['date'].str[:10]
    df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')
    df = df.sort_values(['date'], ascending=[True]).reset_index(drop=True).sort_values(['date'], ascending=[False])
    vol_asset ='Volume %s' %(asset)
    vol_pair ='Volume %s' %(pair)
    df = df.rename(columns={vol_asset:'Volume asset', vol_pair:'Volume pair'})
    df = df[['date', 'Volume asset','Volume pair', 'close']]
    df['volume_rate'] = df['Volume pair'] / df['Volume asset']
    df['volume_rate_vs_close'] = (df['Volume pair'] / df['Volume asset'])/df['close']

    return(df)

#########################################################


def squeez_momentum_indicator(df, column_close='close', column_date='date', column_high='high', column_low='low',
                              length = 20, mult = 2, length_KC = 20, mult_KC = 1.5):

    df = df.sort_values([column_date], ascending=[False]).reset_index(drop=True).sort_values([column_date], ascending=[True])

    df = df.copy()
    df=df[[column_date, column_close, column_high, column_low]]
    # calculate BB
    m_avg = df[column_close].rolling(window=length).mean()
    m_std = df[column_close].rolling(window=length).std(ddof=0)
    df['upper_BB'] = m_avg + mult * m_std
    df['lower_BB'] = m_avg - mult * m_std

    # calculate true range
    df['tr0'] = abs(df[column_high] - df[column_low])
    df['tr1'] = abs(df[column_high] - df[column_close].shift())
    df['tr2'] = abs(df[column_low] - df[column_close].shift())
    df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)

    # calculate KC
    range_ma = df['tr'].rolling(window=length_KC).mean()
    df['upper_KC'] = m_avg + range_ma * mult_KC
    df['lower_KC'] = m_avg - range_ma * mult_KC

    # calculate bar value
    highest = df[column_high].rolling(window = length_KC).max()
    lowest = df[column_low].rolling(window = length_KC).min()
    m1 = (highest + lowest)/2
    df['value'] = (df[column_close] - (m1 + m_avg)/2)
    fit_y = np.array(range(0,length_KC))
    df['value'] = df['value'].rolling(window = length_KC).apply(
        lambda x: np.polyfit(fit_y, x, 1)[0] * (length_KC-1) + np.polyfit(fit_y, x, 1)[1], raw=True
    )

    # check for 'squeeze'
    df['squeeze_on'] = (df['lower_BB'] > df['lower_KC']) & (df['upper_BB'] < df['upper_KC'])
    df['squeeze_off'] = (df['lower_BB'] < df['lower_KC']) & (df['upper_BB'] > df['upper_KC'])

    #drop main features

    df = df[[column_date, 'value', 'squeeze_on', 'squeeze_off', 'tr0','tr1','tr2','tr']]

    return df

####################################################


def squeez_momentum_indicator(df, column_close='close', column_date='date', column_high='high', column_low='low',
                              length = 20, mult = 2, length_KC = 20, mult_KC = 1.5):

    df = df.sort_values([column_date], ascending=[False]).reset_index(drop=True).sort_values([column_date], ascending=[True])

    df = df.copy()
    df=df[[column_date, column_close, column_high, column_low]]
    # calculate BB
    m_avg = df[column_close].rolling(window=length).mean()
    m_std = df[column_close].rolling(window=length).std(ddof=0)
    df['upper_BB'] = m_avg + mult * m_std
    df['lower_BB'] = m_avg - mult * m_std

    # calculate true range
    df['tr0'] = abs(df[column_high] - df[column_low])
    df['tr1'] = abs(df[column_high] - df[column_close].shift())
    df['tr2'] = abs(df[column_low] - df[column_close].shift())
    df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)

    # calculate KC
    range_ma = df['tr'].rolling(window=length_KC).mean()
    df['upper_KC'] = m_avg + range_ma * mult_KC
    df['lower_KC'] = m_avg - range_ma * mult_KC

    # calculate bar value
    highest = df[column_high].rolling(window = length_KC).max()
    lowest = df[column_low].rolling(window = length_KC).min()
    m1 = (highest + lowest)/2
    df['value'] = (df[column_close] - (m1 + m_avg)/2)
    fit_y = np.array(range(0,length_KC))
    df['value'] = df['value'].rolling(window = length_KC).apply(
        lambda x: np.polyfit(fit_y, x, 1)[0] * (length_KC-1) + np.polyfit(fit_y, x, 1)[1], raw=True
    )

    # check for 'squeeze'
    df['squeeze_on'] = (df['lower_BB'] > df['lower_KC']) & (df['upper_BB'] < df['upper_KC'])
    df['squeeze_off'] = (df['lower_BB'] < df['lower_KC']) & (df['upper_BB'] > df['upper_KC'])

    #drop main features

    df = df[[column_date, 'value', 'squeeze_on', 'squeeze_off', 'tr0','tr1','tr2','tr']]

    return df

def calculate_diferentials_resistances(lista, valor_buscado):
    last_support = "UNKNOWN"
    second_last_support = "UNKNOWN"
    last_resistence= "UNKNOWN"
    second_last_resistence = "UNKNOWN"
    posicion = bisect.bisect_left(lista, valor_buscado)
    if len(lista) == 0:
        print("no hay valores encontrados en: ", valor_buscado, lista)
    elif len(lista) == 1:
        if posicion == 0:
            last_support = "NOT FOUND"
            second_last_support = "NOT FOUND"
            second_last_resistence = "NOT FOUND"
            last_resistence= lista[0]
        elif posicion==1:
            second_last_support = "NOT FOUND"
            last_resistence= "NOT FOUND"
            second_last_resistence = "NOT FOUND"
            last_support = lista[0]
    elif len(lista) == 2:
        if posicion == 0:
            last_support = "NOT FOUND"
            second_last_support = "NOT FOUND"
            last_resistence= lista[posicion]
            second_last_resistence = lista[posicion+1]
        elif posicion == 1:
            second_last_support = "NOT FOUND"
            second_last_resistence = "NOT FOUND"
            last_support = lista[posicion-1]
            last_resistence= lista[posicion]
        elif posicion == 2:
            last_support = "NOT FOUND"
            second_last_support = "NOT FOUND"
            last_resistence= "NOT FOUND"
            second_last_resistence = "NOT FOUND"
            last_support = lista[posicion-1]
            second_last_support = lista[posicion-2]
    elif len(lista) == 3:
        if posicion == 0:
            last_support = "NOT FOUND"
            second_last_support = "NOT FOUND"
            last_resistence= lista[posicion]
            second_last_resistence = lista[posicion+1]
        elif posicion == 1:
            second_last_support = "NOT FOUND"
            last_support = lista[posicion-1]
            last_resistence= lista[posicion]
            second_last_resistence = lista[posicion+1]
        elif posicion == 2:
            second_last_resistence = "NOT FOUND"
            second_last_support = lista[posicion-2]
            last_support = lista[posicion-1]
            last_resistence= lista[posicion]
        elif posicion == 3:
            last_resistence= "NOT FOUND"
            second_last_resistence = "NOT FOUND"
            last_support = lista[posicion-1]
            second_last_support = lista[posicion-2]
    elif len(lista) > 3:
        if posicion == 0:
            last_support= "NOT FOUND"
            second_last_support = "NOT FOUND"
            last_resistence= lista[posicion]
            second_last_resistence = lista[posicion+1]
        elif posicion == len(lista):
            last_resistence= "NOT FOUND"
            second_last_resistence = "NOT FOUND"
            last_support = lista[posicion-1]
            second_last_support = lista[posicion-2]
        elif posicion == 1:
            second_last_support = "NOT FOUND"
            last_support = lista[posicion-1]
            last_resistence= lista[posicion]
            second_last_resistence = lista[posicion+1]
        elif posicion == len(lista)-1:
            second_last_resistence = "NOT FOUND"
            last_support = lista[posicion-1]
            second_last_support = lista[posicion-2]
            last_resistence= lista[posicion]
        else:
            last_support = lista[posicion-1]
            second_last_support = lista[posicion-2]
            last_resistence= lista[posicion]
            second_last_resistence = lista[posicion+1]
    list_of_resistences=[second_last_support, last_support, last_resistence, second_last_resistence]
    return(list_of_resistences)

def last_support_unification(last_support, \
                             second_last_support_200, last_support_200, \
                             last_resistence, second_last_resistence):
    list_of_supports = [second_last_support_200, last_support_200]
    list_minimum=[]
    if last_support == 'NOT FOUND':
        if last_resistence != 'NOT FOUND':
            for i in list_of_supports:
                if i != 'NOT FOUND':
                    if i < int(last_resistence):
                        list_minimum.append(i)
            if len(list_minimum)>0:
                out =  max(list_minimum)
            else:
                out=-1
        else:
            for i in list_of_supports:
                if i != 'NOT FOUND':
                    list_minimum.append(i)
            if len(list_minimum)>0:
                out =  (max(list_minimum))
            else:
                out=-2
    else:
        out = last_support
    return out

def second_last_support_unification(new_last_support, second_last_support, last_support, \
                             second_last_support_200, last_support_200, \
                             last_resistence, second_last_resistence):
    list_of_supports = [second_last_support_200, last_support_200]
    list_minimum=[]
    if second_last_support == 'NOT FOUND':
        if last_resistence != 'NOT FOUND':
            for i in list_of_supports:
                if i != 'NOT FOUND':
                    if i < new_last_support and  i < int(last_resistence):
                        list_minimum.append(i)
            if len(list_minimum)>0:
                out =  max(list_minimum)
            else:
                out=-1
        else:
            for i in list_of_supports:
                if i != 'NOT FOUND':
                     if i < new_last_support:
                            list_minimum.append(i)
            if len(list_minimum)>0:
                out =  (max(list_minimum))
            else:
                out=-1
    else:
        out = second_last_support
    return out

def last_resistence_unification(last_resistence, \
                             second_last_resistence_200, last_resistence_200, \
                             last_support):
    list_of_resistences = [second_last_resistence_200, last_resistence_200]
    list_minimum=[]
    if last_resistence == 'NOT FOUND':
        if last_support != 'NOT FOUND':
            for i in list_of_resistences:
                if i != 'NOT FOUND':
                    if i > last_support:
                        list_minimum.append(i)
            if len(list_minimum)>0:
                out =  min(list_minimum)
            else:
                out=-1
        else:
            for i in list_of_resistences:
                if i != 'NOT FOUND':
                    list_minimum.append(i)
            if len(list_minimum)>0:
                out =  (min(list_minimum))
            else:
                out=-2
    else:
        out = last_resistence
    return out

def second_last_resistence_unification(new_last_resistence, second_last_resistence, last_resistence, \
                             second_last_resistence_200, last_resistence_200, last_support):
    list_of_resistences = [second_last_resistence_200, last_resistence_200]
    list_minimum=[]
    if second_last_resistence == 'NOT FOUND':
        if last_support != 'NOT FOUND':
            for i in list_of_resistences:
                if i != 'NOT FOUND':
                    if i > new_last_resistence and  i > last_support:
                        list_minimum.append(i)
            if len(list_minimum)>0:
                out =  min(list_minimum)
            else:
                out=-1
        else:
            for i in list_of_resistences:
                if i != 'NOT FOUND':
                     if i > new_last_resistence:
                            list_minimum.append(i)
            if len(list_minimum)>0:
                out =  (min(list_minimum))
            else:
                out=-2
    else:
        out = second_last_resistence
    return out

def fill_support_not_found_values_in_sr(df, column, measure_column='close', value_to_clean=-1):
    df_out=df.copy()
    for i in range(len(df_out)):
        if df_out.iloc[i][column] == value_to_clean and df_out.iloc[i][measure_column] != value_to_clean:
            for var in range(i+1,len(df_out)):
                if (df_out.iloc[var][column] > -1) and (df_out.iloc[var][column] < df_out.iloc[i][measure_column]):
                    df_out.at[i, column] = df_out.iloc[var][column]
                    break
    return df_out

def fill_resistence_not_found_values_in_sr(df, column, measure_column='close', value_to_clean=-1):
    df_out=df.copy()
    for i in range(len(df_out)):
        if df_out.iloc[i][column] == value_to_clean and df_out.iloc[i][measure_column] != value_to_clean:
            for var in range(i+1,len(df_out)):
                if (df_out.iloc[var][column] > -1) and (df_out.iloc[var][column] > df_out.iloc[i][measure_column]):
                    df_out.at[i, column] = df_out.iloc[var][column]
                    break
    return df_out

def get_supports_and_resistences(df, close_column='close', date_column='date'):
    df_sr  =supports_and_resistences(df, look_back=100)
    df_sr_200 = supports_and_resistences_more_days(df, 200, window_=10)
    df_close = df[['date', 'close']].copy()
    df_sr_total = pd.merge(df_sr, df_sr_200, on='date', how='left')
    df_sr_total.fillna('NOT FOUND', inplace=True)
    df_sr_total = pd.merge(df_sr_total,df_close,on='date',how='inner')
    df_sr_total['new_last_support'] = df_sr_total[['last_support', 'second_last_support_200', 'last_support_200',\
                                          'last_resistence','second_last_resistence']].apply(lambda x : last_support_unification(*x), axis=1)
    df_sr_total['new_second_last_support'] = df_sr_total[['new_last_support', 'second_last_support','last_support', 'second_last_support_200', 'last_support_200',\
                                          'last_resistence','second_last_resistence']].apply(lambda x : second_last_support_unification(*x), axis=1)
    df_sr_total['new_last_resistence'] = df_sr_total[['last_resistence', 'second_last_resistence_200', 'last_resistence_200',\
                                          'last_support']].apply(lambda x : last_resistence_unification(*x), axis=1)
    df_sr_total['new_second_last_resistence'] = df_sr_total[['new_last_resistence', 'second_last_resistence', 'last_resistence', 'second_last_resistence_200', 'last_resistence_200',\
                                          'last_support']].apply(lambda x : second_last_resistence_unification(*x), axis=1)
    df_sr_total = df_sr_total[['date', 'close', 'new_second_last_resistence', 'new_last_resistence', 'new_second_last_support', 'new_last_support']]
    df_filled = df_sr_total.copy()
    df_filled = fill_support_not_found_values_in_sr(df_filled, 'new_last_support')
    df_filled = fill_support_not_found_values_in_sr(df_filled, 'new_second_last_support', measure_column='new_last_support')
    df_filled = fill_resistence_not_found_values_in_sr(df_filled, 'new_last_resistence')
    df_filled = fill_resistence_not_found_values_in_sr(df_filled, 'new_second_last_resistence', measure_column='new_last_resistence')
    df_filled = df_filled.rename(columns={'new_second_last_resistence':'second_last_resistence',\
                                         'new_last_resistence':'last_resistence',\
                                         'new_last_support':'last_support',\
                                         'new_second_last_support':'second_last_support'})
    df_out = df_filled

    for i in range(len(df_out)):
        df_it_out = df_out[i:len(df_out)]
        maximum_value = df_it_out.close.max()
        minimum_value = df_it_out.close.min()
        if df_out[['last_support']].iloc[i][0]==-1:
            df_out.at[i, 'last_support'] = minimum_value
        if df_out[['second_last_support']].iloc[i][0]==-1:
            df_out.at[i, 'second_last_support'] = minimum_value
        if df_out[['last_resistence']].iloc[i][0]==-1:
            df_out.at[i, 'last_resistence'] = maximum_value
        if df_out[['second_last_resistence']].iloc[i][0]==-1:
            df_out.at[i, 'second_last_resistence'] = maximum_value
    df_out =df_out.drop(columns=close_column, axis=1)

    return df_out

def getHigherLows(data: np.array, order=5, K=2):
    '''
    Finds consecutive higher lows in price pattern.
    Must not be exceeded within the number of periods indicated by the width
    parameter for the value to be confirmed.
    K determines how many consecutive lows need to be higher.
    '''
    # Get lows
    low_idx = argrelextrema(data, np.less, order=order)[0]
    lows = data[low_idx]
    # Ensure consecutive lows are higher than previous lows
    extrema = []
    ex_deque = deque(maxlen=K)
    for i, idx in enumerate(low_idx):
        if i == 0:
            ex_deque.append(idx)
            continue
        if lows[i] < lows[i-1]:
            ex_deque.clear()

        ex_deque.append(idx)
        if len(ex_deque) == K:
            extrema.append(ex_deque.copy())

    return extrema

def getLowerHighs(data: np.array, order=5, K=2):
    '''
    Finds consecutive lower highs in price pattern.
    Must not be exceeded within the number of periods indicated by the width
    parameter for the value to be confirmed.
    K determines how many consecutive highs need to be lower.
    '''
    # Get highs
    high_idx = argrelextrema(data, np.greater, order=order)[0]
    highs = data[high_idx]
    # Ensure consecutive highs are lower than previous highs
    extrema = []
    ex_deque = deque(maxlen=K)
    for i, idx in enumerate(high_idx):
        if i == 0:
            ex_deque.append(idx)
            continue
        if highs[i] > highs[i-1]:
            ex_deque.clear()

        ex_deque.append(idx)
        if len(ex_deque) == K:
            extrema.append(ex_deque.copy())

    return extrema

def getHigherHighs(data: np.array, order=5, K=2):
    '''
    Finds consecutive higher highs in price pattern.
    Must not be exceeded within the number of periods indicated by the width
    parameter for the value to be confirmed.
    K determines how many consecutive highs need to be higher.
    '''
    # Get highs
    high_idx = argrelextrema(data, np.greater, order=5)[0]
    highs = data[high_idx]
    # Ensure consecutive highs are higher than previous highs
    extrema = []
    ex_deque = deque(maxlen=K)
    for i, idx in enumerate(high_idx):
        if i == 0:
            ex_deque.append(idx)
            continue
        if highs[i] < highs[i-1]:
            ex_deque.clear()

        ex_deque.append(idx)
        if len(ex_deque) == K:
            extrema.append(ex_deque.copy())

    return extrema

def getLowerLows(data: np.array, order=5, K=2):
    '''
    Finds consecutive lower lows in price pattern.
    Must not be exceeded within the number of periods indicated by the width
    parameter for the value to be confirmed.
    K determines how many consecutive lows need to be lower.
    '''
    # Get lows
    low_idx = argrelextrema(data, np.less, order=order)[0]
    lows = data[low_idx]
    # Ensure consecutive lows are lower than previous lows
    extrema = []
    ex_deque = deque(maxlen=K)
    for i, idx in enumerate(low_idx):
        if i == 0:
            ex_deque.append(idx)
            continue
        if lows[i] > lows[i-1]:
            ex_deque.clear()

        ex_deque.append(idx)
        if len(ex_deque) == K:
            extrema.append(ex_deque.copy())

        return extrema

# no se usan
def getHHIndex(data: np.array, order=150, K=2):
    extrema = getHigherHighs(data, order, K)
    print(extrema)
    idx = np.array([i[-1] + order for i in extrema])
    print(idx)
    return idx[np.where(idx<len(data))]
def getLHIndex(data: np.array, order=5, K=2):
    extrema = getLowerHighs(data, order, K)
    idx = np.array([i[-1] + order for i in extrema])
    return idx[np.where(idx<len(data))]
def getLLIndex(data: np.array, order=5, K=2):
    extrema = getLowerLows(data, order, K)
    idx = np.array([i[-1] + order for i in extrema])
    return idx[np.where(idx<len(data))]
def getHLIndex(data: np.array, order=5, K=2):
    extrema = getHigherLows(data, order, K)
    idx = np.array([i[-1] + order for i in extrema])
    return idx[np.where(idx<len(data))]


def get_hh_divergences_paterns(close_t, function, a, days=5):
    if function == getHigherHighs:
        inc=0
    elif function == getLowerLows:
        inc=10000000000000000000000000000000000000
    elif function == getHigherLows:
        inc=10000000000000000000000000000000000000
    elif function == getLowerHighs:
        inc=0
    #treatement
    close_t = list(close_t)
    close_t.append(inc)
    close_t = np.asarray(close_t)
    hhp = function(close_t)
    hhp_temp=0

    try:
        hhp_temp = hhp[-1][1]
    except:
        pass
    if a-hhp_temp < days:
        hh_value=1
    else:
        hh_value=0
    # output hh_value
    return(hh_value)

def get_divergences_points(df, indicator='close', date_column='date', days_init=5):
    df = df.sort_values([date_column], ascending=[False]).reset_index(drop=True).sort_values([date_column], ascending=[True]).copy()
    df = df[[date_column, indicator]]
    df = df.dropna()
    close = df[indicator].values
    dates = df[date_column].values
    hh = []
    hl = []
    ll = []
    lh = []
    for i, k in enumerate(dates):
        # input
        close_temp=close[:i+1]
        # 1.1 higher highs
        hh.append(get_hh_divergences_paterns(close_temp, getHigherHighs, i))
        #1.2 lower lows
        ll.append(get_hh_divergences_paterns(close_temp, getLowerLows, i))
        #1.3 Higher lows
        hl.append(get_hh_divergences_paterns(close_temp, getHigherLows, i))
        #1.4 Lower Highs
        lh.append(get_hh_divergences_paterns(close_temp, getLowerHighs, i))

    for i in range(days_init):
        hh[i]=0
        ll[i]=0
        hl[i]=0
        lh[i]=0
    data = pd.DataFrame(columns = [date_column, 'hh_'+indicator, 'll_'+indicator, 'hl_'+indicator, 'lh_'+indicator])
    df_final = data.append(pd.DataFrame({date_column : dates, 'hh_'+indicator:hh, 'll_'+indicator:ll, 'hl_'+indicator:hl, 'lh_'+indicator:lh}))

    return(df_final)



def add_derivates(df, feature, periods=1):
    df_new = df.copy()
    diff_1 = df[[feature]].diff(periods=periods).rename(columns={feature: feature +"_diff1"})
    df_new = pd.concat([df_new, diff_1], axis=1).dropna()
    del df
    return df_new

def get_ribbons(df, sma_low, sma_high, list_of_features, date_column='date'):
    df_out=df.copy()
    list_of_columns = [date_column] + list_of_features
    df_out=df_out[list_of_columns]
    list_of_sma = []
    list_of_ribbon = []
    for i in list_of_features:
        sma_low_ = i + '_sma_' + str(sma_low)
        list_of_sma.append(sma_low_)
        df_out[sma_low_] = pd.Series.to_frame(ta.sma(df_out[i], length=sma_low))
        sma_high_ = i + '_sma_' + str(sma_high)
        list_of_sma.append(sma_high_)
        df_out[sma_high_] = pd.Series.to_frame(ta.sma(df_out[i], length=sma_high))
        ribbon_ = 'ribbon_' + i + '_' +  str(sma_low) + '_' + str(sma_high)
        list_of_ribbon.append(ribbon_)
        df_out[ribbon_] = (df_out[sma_low_] - df_out[sma_high_])
    df_out = df_out.dropna()
    return df_out

def get_ribbons_30_60(df, list_of_names_df):
    df_final=df.copy()
    list_of_sma = []
    list_of_ribbon = []
    for i in list_of_names_df:
        sma_30 = i + '_sma_30'
        list_of_sma.append(sma_30)
        df_final[sma_30] = pd.Series.to_frame(ta.sma(df_final[i], length=30))
        sma_60 = i + '_sma_60'
        list_of_sma.append(sma_60)
        df_final[sma_60] = pd.Series.to_frame(ta.sma(df_final[i], length=60))
        ribbon_30_60 = 'ribbon_' + i + '_30_60'
        list_of_ribbon.append(ribbon_30_60)
        df_final[ribbon_30_60] = (df_final[sma_30] - df_final[sma_60])
    return df_final

def create_linear_regression_of_maximums(df_in, feature_column, date_column='date'):

    df_out = df_in[[date_column, feature_column]]
    if date_column != 'date':
        df_out = df_out.rename(columns={date_column:'date'})
    date_column='date'
    first_date_str = df_out.iloc[0].date
    first_date = first_date_str.to_pydatetime().date()
    df_out[df_out[date_column]==first_date_str].iloc[0][1]#.CPTRA_sma_30
    maximum_date_CPTRA = {}
    today = date.today()
    number_of_years = today.year - first_date.year
    low_date = first_date
    maximum_value=0
    list_of_dates=[]
    list_of_values=[]

    for i in range(number_of_years+1):
        low_date =  first_date_str + dt.timedelta(days = 365*i)
        high_date =  first_date_str + dt.timedelta(days = 365*(i+1))
        df_filtered = df_out[(df_out[date_column] > low_date) &  (df_out[date_column] <= high_date)]
        maximum = df_filtered[[feature_column]].max()[0]#.CPTRA_sma_30
        if maximum > maximum_value:
            maximum_value = maximum
            list_of_dates.append(df_filtered[df_filtered[feature_column]==df_filtered[feature_column].max()].reset_index().iloc[0].date)
            list_of_values.append(maximum)

    dataframe_dates_maximum = pd.DataFrame(
        {date_column: list_of_dates,
        feature_column: list_of_values
        })

    total_days = (today - first_date).days
    # https://laid-back-scientist.com/en/fill-datetime
    df_ = dataframe_dates_maximum.set_index(date_column)
    df_ = df_.asfreq(freq='1440min')
    df_fill = df_.reset_index()

    # https://mohammadimranhasan.com/linear-regression-of-time-series-data-with-pandas-library-in-python/
    df_dates = df_out[[date_column]].copy()
    df_new = pd.merge(df_dates, dataframe_dates_maximum, how='left', on=date_column)
    df_fill = df_new.copy()
    df_fill =df_fill.set_index(date_column)
    y=np.array(df_fill[feature_column].dropna().values, dtype=float)
    x=np.array(pd.to_datetime(df_fill[feature_column].dropna()).index.values, dtype=float)
    x_tot = np.array(pd.to_datetime(df_fill[feature_column]).index.values, dtype=float)
    slope, intercept, r_value, p_value, std_err =sp.linregress(x,y)
    xf = np.linspace(min(x_tot),max(x_tot),len(df_fill))
    xf1 = xf.copy()
    xf1 = pd.to_datetime(xf1)
    yf = (slope*xf)+intercept
    x_for_df = xf1.to_list()
    y_for_df = np.array(yf).tolist()
    name_of_feature = 'linear_regression_max_' + feature_column
    df_linear_regression_max = pd.DataFrame(
        {date_column: x_for_df,
        name_of_feature: y_for_df
        })
    df_linear_regression_max = df_linear_regression_max[df_linear_regression_max['linear_regression_max_' + feature_column]>0]
    df_linear_regression_max[date_column] = pd.to_datetime(df_linear_regression_max[date_column]).dt.date
    df_linear_regression_max[date_column] = pd.to_datetime(df_linear_regression_max[date_column])
    #df_linear_regression_max[name_of_feature] = df_linear_regression_max[name_of_feature].clip(lower=0)

    return df_linear_regression_max

def create_linear_regression_of_minimums(df_in, feature_column, date_column='date'):

    df_out = df_in[[date_column,feature_column]].copy()

    first_date_str = df_out.iloc[0].date
    first_date = first_date_str.to_pydatetime().date()
    df_out[df_out[date_column]==first_date_str].iloc[0].CPTRA_sma_30
    maximum_date_CPTRA = {}
    today = date.today()
    number_of_years = today.year - first_date.year
    low_date = first_date
    maximum_value=0
    list_of_dates=[]
    list_of_values=[]

    for i in range(number_of_years+1):
        low_date =  first_date_str + dt.timedelta(days = 365*i)
        high_date =  first_date_str + dt.timedelta(days = 365*(i+1))
        df_filtered = df_out[(df_out[date_column] > low_date) &  (df_out[date_column] <= high_date)]
        minimum = df_filtered[[feature_column]].min().CPTRA_sma_30
        try:
            list_of_dates.append(df_filtered[df_filtered[feature_column]==df_filtered[feature_column].min()].reset_index().iloc[0].date)
            list_of_values.append(minimum)
        except:
            pass

    # Now get minimum absolutes

    list_of_absolutes_dates=[]
    list_of_absolutes_values=[]
    count=0
    length_data = len(list_of_values)
    maximum_of_minimum = list_of_values[length_data-1]
    list_of_absolutes_values.append(list_of_values[length_data-1])
    list_of_absolutes_dates.append(list_of_dates[length_data-1])
    for i in reversed(list_of_values):
        count+=1
        if i < maximum_of_minimum:
            maximum_of_minimum = i
            list_of_absolutes_values.append(i)
            list_of_absolutes_dates.append(list_of_dates[length_data-count])

    dataframe_dates_minimum = pd.DataFrame(
        {date_column: list_of_absolutes_dates,
        feature_column: list_of_absolutes_values
        })
    dataframe_dates_minimum = dataframe_dates_minimum[:-1] # delete last data
    total_days = (today - first_date).days
    df_ = dataframe_dates_minimum.set_index(date_column)
    df_ = df_.asfreq(freq='1440min')
    df_fill = df_.reset_index()
    df_dates = df_out[[date_column]]
    df_new = pd.merge(df_dates, dataframe_dates_minimum, how='left', on=date_column)
    df_fill = df_new.copy()
    df_fill =df_fill.set_index(date_column)

    # https://mohammadimranhasan.com/linear-regression-of-time-series-data-with-pandas-library-in-python/
    df_dates = df_out[[date_column]].copy()
    df_new = pd.merge(df_dates, dataframe_dates_minimum, how='left', on=date_column)
    df_fill = df_new.copy()
    df_fill =df_fill.set_index(date_column)
    y=np.array(df_fill[feature_column].dropna().values, dtype=float)
    x=np.array(pd.to_datetime(df_fill[feature_column].dropna()).index.values, dtype=float)
    x_tot = np.array(pd.to_datetime(df_fill[feature_column]).index.values, dtype=float)
    slope, intercept, r_value, p_value, std_err =sp.linregress(x,y)
    xf = np.linspace(min(x_tot),max(x_tot),len(df_fill))
    xf1 = xf.copy()
    xf1 = pd.to_datetime(xf1)
    yf = (slope*xf)+intercept
    x_for_df = xf1.to_list()
    y_for_df = np.array(yf).tolist()
    name_of_feature = 'linear_regression_min_' + feature_column
    df_linear_regression_min = pd.DataFrame(
        {date_column: x_for_df,
        name_of_feature : y_for_df
        })
    df_linear_regression_min = df_linear_regression_min[df_linear_regression_min['linear_regression_min_' + feature_column]>0]
    df_linear_regression_min[date_column] = pd.to_datetime(df_linear_regression_min[date_column]).dt.date
    df_linear_regression_min[date_column] = pd.to_datetime(df_linear_regression_min[date_column])
    df_linear_regression_min[name_of_feature] = df_linear_regression_min[name_of_feature].clip(lower=0)
    return df_linear_regression_min

def create_list_of_columns(df, datecolumn='date'):
    list_out = []
    for i in df.columns:
        if i != datecolumn:
            list_out.append(i)
    return list_out

def obtener_combinaciones(lista):
    resultados = []
    for i in range(1, len(lista) + 1):
        combos = combinations(lista, i)
        resultados.extend(list(combos))

    # Convertir cada tupla en una lista
    resultados = [list(combo) for combo in resultados]
    return resultados

def create_model(X_train, input_layers = 4, units=200, drop=0.1):
    model = Sequential()
    for i in range(input_layers-1):
        model.add(LSTM(units = units, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
        model.add(Dropout(drop))
    model.add(Dropout(drop))
    model.add(LSTM(units = units))
    #model.add(Dense(1)) #y_train.shape[-1])
    model.add(Dense(1, activation='sigmoid'))
    return(model)

def create_dataset(dataset, dataset_y, look_back, look_forward):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-look_forward):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset_y[(i + look_back+ look_forward-1):(i + look_back + look_forward), 0]) #ojo look_forward primero
    return np.array(dataX), np.array(dataY)

def create_dataset_official(dataset, dataset_y, look_back):
    dataX, dataY = [], []
    for i in range(look_back,len(dataset)):
        a = dataset[i-look_back:i]
        dataX.append(a)
        dataY.append(dataset_y[i])
    return np.array(dataX), np.array(dataY)

def create_dataset_X_y(dataset_X, dataset_y, look_back, test_size):
    pos = int(round(len(dataset_X) * (1-test_size)))
    data_train_X, data_train_y, data_test_X, data_test_y = [], [], [], []
    for i in range(look_back,pos):
        a = dataset_X[i-look_back:i]
        data_train_X.append(a)
        data_train_y.append(dataset_y[i])
    for i in range(pos,len(dataset_X)):
        a = dataset_X[i-look_back:i]
        data_test_X.append(a)
        data_test_y.append(dataset_y[i])
    return np.array(data_train_X), np.array(data_train_y), np.array(data_test_X), np.array(data_test_y)

def split_data_dates(dates_, test_size):
        pos = int(round(len(dates_) * (1-test_size)))
        dates_train = dates_[:pos]
        dates_test = dates_[pos:]
        return dates_train, dates_test

def split_data(X, y, test_size):
        pos = int(round(len(X) * (1-test_size)))
        X_train, y_train = X[:pos], y[:pos]
        X_test, y_test = X[pos:], y[pos:]
        return X_train, y_train, X_test, y_test

def test_lstm(df_in, capas=4, epochs=20, look_back=10, look_forward=0, batch_size=20, split=0.2,
              drop=0.1, optimizador='sgd', initial_days=1, days_target=10, column_target = 'target', units=50, column_date='date'):
    #df_lstm = df.sort_index(ascending=False)
    dates = df_in[column_date]
    dates_train, dates_test = split_data_dates(dates, split)

    X=df_in.drop(columns=['target',column_date], axis=1)
    #Y será la clase a predecir

    y=df_in['target']

    #dataset= X.values
    dataset_y = y.values
    df_for_scaled = pd.concat([X,y], axis=1)
    dataset = df_for_scaled.values
    dataset = dataset.reshape(-1,dataset[0].size)
    dataset = dataset.astype("float32")

    #dataset_y = dataset_y.reshape(-1,1)
    dataset_y = dataset_y.astype("float32")

    # scaling datase
    scaler = MinMaxScaler(feature_range=(0,1))
    dataset = scaler.fit_transform(dataset)
    #dataset_y = scaler.fit_transform(dataset_y)
    #X, y = create_dataset(dataset, dataset_y, look_back,look_forward)
    X, y = create_dataset_official(dataset, dataset_y, look_back)
    print("X:{},y:{}".format(X.shape, y.shape))
    X_train, y_train, X_test, y_test = split_data(X, y, split)
    print("X_train:{},y_train:{},X_test:{},y_test{}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))
    # Initialising the LSTM
    model = create_model(X_train, input_layers=capas)
    model.compile(optimizer = optimizador, loss = 'mean_squared_error')
    hist = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

    #### EXECUTING THE MODEL ####
    # predictions
    trainPredict = model.predict(X_train)
    testPredict = model.predict(X_test)

    # reshape
    #trainPredict = scaler.inverse_transform(trainPredict)
    #trainY = scaler.inverse_transform(y_train)
    #testPredict = scaler.inverse_transform(testPredict)
    #testY = scaler.inverse_transform(y_test)
    testY = y_test
    trainY = y_train
    # metrics
    #trainY=trainY.reshape(1,-1)
    #testY = testY.reshape(1,-1)
    # Calcular AUC
    auc_test = roc_auc_score(testY, testPredict)
    auc_train = roc_auc_score(trainY, trainPredict)
    acc_test = accuracy_score(testY, testPredict.round())
    acc_train = accuracy_score(trainY, trainPredict.round())
    return(auc_test, auc_train, acc_test, acc_train, len(X_train), len(X_test), testPredict, dates_test)


def lstm(df_in, ticker, capas=3, epochs=60, look_back=25, batch_size=20, split=0.2,
              drop=0.1, optimizador='sgd', initial_days=1, days_target=10, column_target = 'target', units=50, column_date='date'):
    #df_lstm = df.sort_index(ascending=False)
    dates = df_in['date']
    dates_train, dates_test = split_data_dates(dates, split)
    X=df_in.drop(columns=['target',column_date], axis=1)
    #Y será la clase a predecir
    y=df_in[column_target]
    #dataset= X.values
    dataset_y = y.values
    df_for_scaled = pd.concat([X,y], axis=1)
    dataset = df_for_scaled.values
    dataset = dataset.reshape(-1,dataset[0].size)
    dataset = dataset.astype("float32")
    #dataset_y = dataset_y.reshape(-1,1)
    dataset_y = dataset_y.astype("float32")

    # scaling datase
    scaler = MinMaxScaler(feature_range=(0,1))
    dataset = scaler.fit_transform(dataset)
    #dataset_y = scaler.fit_transform(dataset_y)
    #X, y = create_dataset(dataset, dataset_y, look_back,look_forward)
    X_train, y_train, X_test, y_test = create_dataset_X_y(X, dataset_y, look_back, split)
    print("X_train:{},y_train:{},X_test:{},y_test{}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))
    # Initialising the LSTM
    model = create_model(X_train, input_layers=capas)
    model.compile(optimizer = optimizador, loss = 'mean_squared_error')
    hist = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    #model.save('model_'+ ticker + '.h5')
    #### EXECUTING THE MODEL ####
    # predictions
    trainPredict = model.predict(X_train)
    testPredict = model.predict(X_test)
    # reshape
    #trainPredict = scaler.inverse_transform(trainPredict)
    #trainY = scaler.inverse_transform(y_train)
    #testPredict = scaler.inverse_transform(testPredict)
    #testY = scaler.inverse_transform(y_test)
    testY = y_test
    trainY = y_train
    # metrics
    #trainY=trainY.reshape(1,-1)
    #testY = testY.reshape(1,-1)
    # Calcular AUC
    auc_test = roc_auc_score(testY, testPredict)
    auc_train = roc_auc_score(trainY, trainPredict)
    acc_test = accuracy_score(testY, testPredict.round())
    acc_train = accuracy_score(trainY, trainPredict.round())
    testPredict_flat = [val[0] for val in testPredict]
    # Crear un DataFrame combinando 'testPredict' y 'dates_test'
    data = {'dates_test': dates_test, 'testPredict': testPredict_flat, 'testY':testY}
    df_out = pd.DataFrame(data)
    return(auc_test, auc_train, acc_test, acc_train, df_out)

columns_to_get_percentage = ['SMA_20',  'EMA_20_OHLC4', 'SMA_55', 'EMA_55_OHLC4', 'EMA_200_OHLC4', 'ISA_9',
       'ISB_26', 'ITS_9', 'IKS_26', 'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0',
       'SQZ_20_2.0_20_1.5', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9']

def create_indicators_percentages(df_in, columns_to_get_percentaje, close_column='close'):
    df_out = df_in.copy()
    for i in columns_to_get_percentaje:
        df_out[i] = 100*((df_out[close_column] - df_out[i])/df_out[close_column])
    return(df_out)



In [ ]:

test_size=0.2
ticker='SAP'
capas=4
epochs=1
look_back=30
batch_size=20
split=0.2
drop=0.1
optimizador='sgd'
initial_days=1
days_target=10
column_target = 'target'
units=50
column_date='date'

columnas = ['epochs', 'layers', 'days_back', 'auc_test', 'auc_train', 'acc_test', 'acc_train']
# Crear un DataFrame vacío con las columnas
df_final = pd.DataFrame(columns=columnas)

list_of_companies = ['SAP', 'RACE', 'DTE', 'AIR', 'ASML', 'SAN', 'NVDA', 'META', 'ELF', 'MDB']
data_train_X, data_train_y, data_test_X, data_test_y = [], [], [], []
for company in list_of_companies:

    file = company + '_PERCENTAGES_AND_CLOSE.csv'
    df_in = pd.read_csv(file)
    df_in['date'] = pd.to_datetime(df_in['date'])
    df_in = df_in.sort_values(by='date')
    df_in = df_in.drop(columns=['symbol'])

    dates = df_in['date']
    #dates_train, dates_test = split_data_dates(dates, split)
    X=df_in.drop(columns=['target',column_date], axis=1)
    #Y será la clase a predecir
    y=df_in[column_target]
    #dataset= X.values
    dataset_y = y.values
    df_for_scaled = pd.concat([X,y], axis=1)
    dataset = df_for_scaled.values
    dataset = dataset.reshape(-1,dataset[0].size)
    dataset = dataset.astype("float32")
    #dataset_y = dataset_y.reshape(-1,1)
    dataset_y = dataset_y.astype("float32")

    dataset_X = X.values
    #dataset_X = dataset_X.reshape(-1,dataset[0].size)
    #dataset_X = dataset_X.astype("float32")
    pos = int(round(len(dataset_X) * (1-test_size)))

    # scaling datase
    scaler = MinMaxScaler(feature_range=(0,1))
    dataset = scaler.fit_transform(dataset)
    dataset_X = scaler.fit_transform(dataset_X)

    for i in range(look_back,pos):
        a = dataset_X[i-look_back:i]
        data_train_X.append(a)
        data_train_y.append(dataset_y[i])
    for i in range(pos,len(dataset_X)):
        a = dataset_X[i-look_back:i]
        data_test_X.append(a)
        data_test_y.append(dataset_y[i])

X_train, y_train, X_test, y_test = np.array(data_train_X), np.array(data_train_y), np.array(data_test_X), np.array(data_test_y)

list_epochs = [20,40,60,100]
list_epochs = [60,100] # quitar esta linea queda 60 epocs y 8
list_capas = [8]
#list_capas = [4,8] # quitar
#list_days_back = [10,20,30]
#list_days_back = [30] #quitar
columnas = ['epochs', 'layers', 'days_back', 'auc_test', 'auc_train', 'acc_test', 'acc_train']
# Crear un DataFrame vacío con las columnas
df_final = pd.DataFrame(columns=columnas)
for epochs in list_epochs:
        for layer in list_capas:
            try:

                model = create_model(X_train, input_layers=layer)
                model.compile(optimizer = optimizador, loss = 'mean_squared_error')
                hist = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
                trainPredict = model.predict(X_train)
                testPredict = model.predict(X_test)
                # reshape
                #trainPredict = scaler.inverse_transform(trainPredict)
                #trainY = scaler.inverse_transform(y_train)
                #testPredict = scaler.inverse_transform(testPredict)
                #testY = scaler.inverse_transform(y_test)
                testY = y_test
                trainY = y_train
                auc_test = roc_auc_score(testY, testPredict)
                auc_train = roc_auc_score(trainY, trainPredict)
                acc_test = accuracy_score(testY, testPredict.round())
                acc_train = accuracy_score(trainY, trainPredict.round())

                data = {'epochs': [epochs], 'layers': [layer], 'days_back': [look_back],
                            'auc_test': [auc_test], 'auc_train': [auc_train], 'acc_test': [acc_test], 'acc_train': [acc_train]}

                df_new = pd.DataFrame(data)
                df_final = pd.concat([df_final, df_new], axis=0, ignore_index=True)
                print(df_final)
            except:
                pass
                print("error")


Epoch 1/100
1094/1094 [==============================] - 22s 10ms/step - loss: 0.2488
Epoch 2/100
1094/1094 [==============================] - 6s 6ms/step - loss: 0.2481
Epoch 3/100
1094/1094 [==============================] - 6s 6ms/step - loss: 0.2477
Epoch 4/100
1094/1094 [==============================] - 6s 5ms/step - loss: 0.2474
Epoch 5/100
1094/1094 [==============================] - 6s 6ms/step - loss: 0.2473
Epoch 6/100
1094/1094 [==============================] - 5s 5ms/step - loss: 0.2472
Epoch 7/100
1094/1094 [==============================] - 6s 6ms/step - loss: 0.2470
Epoch 8/100
1094/1094 [==============================] - 6s 5ms/step - loss: 0.2471
Epoch 9/100
1094/1094 [==============================] - 6s 6ms/step - loss: 0.2467
Epoch 10/100
1094/1094 [==============================] - 5s 5ms/step - loss: 0.2469
Epoch 11/100
1094/1094 [==============================] - 6s 6ms/step - loss: 0.2467
Epoch 12/100
1094/1094 [==============================] - 6s 5ms/step - 